In [1]:
import transformers

c:\Python_tool\Anaconda\envs\torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import torch

In [7]:
from IPython.display import display, Markdown, clear_output
def display_answer(query, history = []):
    # resp, history = get_knowledge_based_answer(query=query,
    #                                            vector_store=vector_store,
    #                                            chat_history=history)
    display(Markdown(query))
display_answer(query="ModelWhale 是什么")

ModelWhale 是什么

In [3]:
torch.cuda.is_available()

True

In [4]:
!nvidia-smi

Mon May 22 15:55:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 528.24       Driver Version: 528.24       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| 18%   32C    P8    14W / 215W |      0MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

In [4]:
print(prompt.format(product="colorful socks"))

What is a good name for a company that makes colorful socks?


In [5]:
import os
os.path.isfile('./data/document_corpus.txt')

True

In [5]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("shibing624/text2vec-base-chinese")

model = AutoModel.from_pretrained("shibing624/text2vec-base-chinese")

c:\Python_tool\Anaconda\envs\torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Python_tool\Anaconda\envs\torch\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Opti7080\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this artic

In [6]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


In [8]:
import torch
sentences = ['如何更换花呗绑定银行卡', '花呗更改绑定银行卡']
# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)
# Perform pooling. In this case, mean pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
print("Sentence embeddings:")
print(sentence_embeddings)

Sentence embeddings:
tensor([[-4.4399e-04, -2.9735e-01,  8.5790e-01,  ..., -5.2770e-01,
         -1.4316e-01, -1.0008e-01],
        [ 6.5362e-01, -7.6667e-02,  9.5962e-01,  ..., -6.0122e-01,
         -1.6797e-03,  2.1458e-01]])


In [3]:
from math import sin

@nb.vectorize()
def nb_vec_sin(a):
    return sin(a)

In [48]:
from text2vec import SentenceModel, cos_sim
import time
import numpy as np
import torch
a = np.random.randn(100000,768)
b = np.random.randn(1,768)
start_time = time.time()
c = cos_sim(a,b)
duration_1 = time.time()-start_time
start_time = time.time()
print(nb_vec_sin(a))
duration_2 = time.time()-start_time

a = torch.tensor(a).to('cuda')
b = torch.tensor(b).to('cuda')
s_time = time.time()
c_ = cos_sim(a,b)
duration_3 = time.time()-s_time

[[-0.25494302  0.35510558  0.65787946 ...  0.23777344  0.89951332
  -0.69283581]
 [-0.21394402 -0.02146462  0.73679789 ...  0.53819302  0.95069866
  -0.97943308]
 [ 0.73773584  0.5728007   0.56698357 ...  0.6696758  -0.91263461
  -0.87383107]
 ...
 [-0.53538724 -0.62236235  0.46455885 ... -0.72469746 -0.43196779
   0.1900131 ]
 [ 0.30064271 -0.95715459  0.9723309  ...  0.68655798  0.98785573
  -0.93550842]
 [-0.49164463 -0.22650817  0.64530563 ...  0.73041068  0.87517238
   0.74054606]]


In [49]:
duration_1,duration_2,duration_3

(0.19809770584106445, 0.3446838855743408, 0.0020568370819091797)

In [22]:
from numba import jit
@jit(nopython=True) 
def t():
    x = 0
    for i in np.arange(5000):
        x += i
    return x
%timeit t() 

1.91 µs ± 2.31 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [25]:
%timeit np.sin(a)
%timeit nb_vec_sin(a)

515 µs ± 768 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
232 µs ± 2.36 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [4]:
from transformers import AutoTokenizer,AutoModel
from argparse import Namespace
model_args = Namespace(do_mlm=None,pooler_type='cls', temp=0.05, mlp_only_train=False,init_embeddings_model=None)
tokenizer = AutoTokenizer.from_pretrained('silk-road/luotuo-bert')
model = AutoModel.from_pretrained('silk-road/luotuo-bert',trust_remote_code=True,model_args=model_args)

In [10]:
import torch
text_left = ['如何更换花呗绑定银行卡']

queries = [
    '如何更换花呗绑定银行卡',
    'A man is eating pasta.',
    'Someone in a gorilla costume is playing a set of drums.',
    'A cheetah chases prey on across a field.']

text_right = [
    '花呗更改绑定银行卡',
    '我什么时候开通了花呗',
    'A man is eating food.',
    'A man is eating a piece of bread.',
    'The girl is carrying a baby.',
    'A man is riding a horse.',
    'A woman is playing violin.',
    'Two men pushed carts through the woods.',
    'A man is riding a white horse on an enclosed ground.',
    'A monkey is playing drums.',
    'A cheetah is running behind its prey.'
]
text_left = queries
inputs = tokenizer(text_left, padding=True, truncation=True, return_tensors="pt")
with torch.no_grad():
    embeddings_left = model(**inputs, output_hidden_states=True, return_dict=True, sent_emb=True).pooler_output
inputs = tokenizer(text_right, padding=True, truncation=True, return_tensors="pt")
with torch.no_grad():
    embeddings_right = model(**inputs, output_hidden_states=True, return_dict=True, sent_emb=True).pooler_output
    
cos_sim_matrix = torch.matmul(embeddings_left, embeddings_right.t())
cos_sim_matrix /= torch.matmul(torch.norm(embeddings_left, dim=1, keepdim=True), torch.norm(embeddings_right, dim=1, keepdim=True).t())
tensor_cpu = cos_sim_matrix.cpu()

In [3]:
queries = [
    '如何更换花呗绑定银行卡',
    'A man is eating pasta.',
    'Someone in a gorilla costume is playing a set of drums.',
    'A cheetah chases prey on across a field.']

text_right = [
    '花呗更改绑定银行卡',
    '我什么时候开通了花呗',
    'A man is eating food.',
    'A man is eating a piece of bread.',
    'The girl is carrying a baby.',
    'A man is riding a horse.',
    'A woman is playing violin.',
    'Two men pushed carts through the woods.',
    'A man is riding a white horse on an enclosed ground.',
    'A monkey is playing drums.',
    'A cheetah is running behind its prey.'
]

In [4]:
model = AutoModel.from_pretrained("THUDM/chatglm-6b-int4",trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm-6b-int4:
- configuration_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm-6b-int4:
- quantization.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm-6b-int4:
- modeling_chatglm.py
- quantization.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


No compiled kernel found.
Compiling kernels : C:\Users\Opti7080\.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\02a065cf2797029c036a02cac30f1da1a9bc49a3\quantization_kernels_parallel.c
Compiling gcc -O3 -fPIC -pthread -fopenmp -std=c99 C:\Users\Opti7080\.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\02a065cf2797029c036a02cac30f1da1a9bc49a3\quantization_kernels_parallel.c -shared -o C:\Users\Opti7080\.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\02a065cf2797029c036a02cac30f1da1a9bc49a3\quantization_kernels_parallel.so
Compile default cpu kernel failed, using default cpu kernel code.
Compiling gcc -O3 -fPIC -std=c99 C:\Users\Opti7080\.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\02a065cf2797029c036a02cac30f1da1a9bc49a3\quantization_kernels.c -shared -o C:\Users\Opti7080\.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\02a065cf2797029c036a02cac30f1da1a9bc49a3\quantization_kern

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (word_embeddings): Embedding(130528, 4096)
    (layers): ModuleList(
      (0-27): 28 x GLMBlock(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attention): SelfAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): QuantizedLinear(in_features=4096, out_features=12288, bias=True)
          (dense): QuantizedLinear(in_features=4096, out_features=4096, bias=True)
        )
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (mlp): GLU(
          (dense_h_to_4h): QuantizedLinear(in_features=4096, out_features=16384, bias=True)
          (dense_4h_to_h): QuantizedLinear(in_features=16384, out_features=4096, bias=True)
        )
      )
    )
    (final_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=4096, out_features=130528, bias=False)
)

In [12]:
from transformers import AutoTokenizer,AutoModel
import torch
model = AutoModel.from_pretrained("THUDM/chatglm-6b-int4", trust_remote_code=True).half().cuda()

tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b-int4", trust_remote_code=True)

c:\Python_tool\Anaconda\envs\torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No compiled kernel found.
Compiling kernels : C:\Users\Opti7080\.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\02a065cf2797029c036a02cac30f1da1a9bc49a3\quantization_kernels_parallel.c
Compiling gcc -O3 -fPIC -pthread -fopenmp -std=c99 C:\Users\Opti7080\.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\02a065cf2797029c036a02cac30f1da1a9bc49a3\quantization_kernels_parallel.c -shared -o C:\Users\Opti7080\.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\02a065cf2797029c036a02cac30f1da1a9bc49a3\quantization_kernels_parallel.so
Compile default cpu kernel failed, using default cpu kernel code.
Compiling gcc -O3 -fPIC -std=c99 C:\Users\Opti7080\.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\02a065cf2797029c036a02cac30f1da1a9bc49a3\quantization_kernels.c -shared -o C:\Users\Opti7080\.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\02a065cf2797029c036a02cac30f1da1a9bc49a3\quantization_kern

In [3]:
context = """本机物理MAC地址是：cc:d9:ac:d8:57:06,
本机电脑名是：DESKTOP-JHRQTGU.lenovo.com
本机IP是：10.176.130.235
本机硬盘序列号是：E823_8FA6_BF53_0001_001B_444A_4423_4BF2.
本机CPU序列号是：BFEBFBFF000A0655
本机主板序列号是：/41JYJ53/CNFCW0007K00LH/
本机bios序列号是：41JYJ53"""
inp = '我的GPU型号是什么'
input_text = f"""基于以下已知信息，简洁和专业的来回答用户的问题。
如果无法从中得到答案，请说 "根据已知信息无法回答该问题" 或 "没有提供足够的相关信息"，不允许在答案中添加编造成分，答案请使用中文。
##已知内容:
{context}
##问题:
{inp}
##回答："""

model.eval()
with torch.no_grad():
        
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to('cuda')
    
    out = model.generate(
        input_ids=input_ids,
        max_length=300,
        temperature=0,
        top_p = 0.95,
        # repetition_penalty = 1.15,
        # stopping_criteria = StoppingCriteriaList([stop_criteria])
        # do_sample = True
    )
    answer = tokenizer.decode(out[0])
    print(answer)
    # item['infer_answer'] = answer
    # print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')

基于以下已知信息,简洁和专业的来回答用户的问题。

##已知内容:
本机物理MAC地址是:cc:d9:ac:d8:57:06,
本机电脑名是:DESKTOP-JHRQTGU.lenovo.com
本机IP是:10.176.130.235
本机硬盘序列号是:E823_8FA6_BF53_0001_001B_444A_4423_4BF2.
本机CPU序列号是:BFEBFBFF000A0655
本机主板序列号是:/41JYJ53/CNFCW0007K00LH/
本机bios序列号是:41JYJ53
##问题:
我的GPU型号是什么
##回答: 根据您提供的信息,您的GPU型号是 NVIDIA GeForce GTX 1050 Ti。


In [33]:
context = """电脑当前音量为50"""
inp = '请帮我降低音量'
input_text = f"""基于以下已知信息，简洁和专业的提取出数字，回答仅需填写一个数字。

##已知内容:
{context}
##问题:
{inp}，请问调整后音量的数字是多少
##回答：
"""

model.eval()
with torch.no_grad():
        
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to('cuda')
    
    out = model.generate(
        input_ids=input_ids,
        max_length=150,
        temperature=0,
        top_p = 0.95,
        # repetition_penalty = 1.3
        # stopping_criteria = StoppingCriteriaList([stop_criteria])
        # do_sample = True
    )
    answer = tokenizer.decode(out[0])
    print(answer)
    # item['infer_answer'] = answer
    # print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')

基于以下已知信息,简洁和专业的提取出数字,回答仅需填写一个数字。

##已知内容:
电脑当前音量为50
##问题:
请帮我降低音量,请问调整后音量的数字是多少
##回答:
 将音量降低到50,则调整后的音量为50。


In [22]:
context = """电脑当前屏幕亮度为80"""
inp = '帮我调低屏幕亮度'
input_text = f"""<用户>：假设你可以直接控制电脑，我将给你两个例子，请按照例子的形式进行回答。
例子：
已知电脑当前屏幕亮度为30，用户需要“帮我调高屏幕亮度”，则直接回答“由于您现在电脑当前亮度为30，已调至60”
已知电脑当前屏幕亮度为50，用户需要“帮我调低屏幕亮度”，则直接回答“由于您现在电脑当前亮度为50，已调至20”
问题：已知{context}，用户需求“{inp}”，请模仿"“由于您现在电脑当前亮度为X，已调至Y”"进行回答：
<bot>:"""

model.eval()
with torch.no_grad():
        
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to('cuda')
    
    out = model.generate(
        input_ids=input_ids,
        max_length=200,
        temperature=0.3,
        top_p = 0.95,
        # repetition_penalty = 1.3
        # stopping_criteria = StoppingCriteriaList([stop_criteria])
        # do_sample = True
    )
    answer = tokenizer.decode(out[0])
    print(answer)
    # item['infer_answer'] = answer
    # print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')

<用户>:假设你可以直接控制电脑,我将给你两个例子,请按照例子的形式进行回答。
例子:
已知电脑当前屏幕亮度为30,用户需要“帮我调高屏幕亮度”,则直接回答“由于您现在电脑当前亮度为30,已调至60”
已知电脑当前屏幕亮度为50,用户需要“帮我调低屏幕亮度”,则直接回答“由于您现在电脑当前亮度为50,已调至20”
问题:已知电脑当前屏幕亮度为80,用户需求“帮我调低屏幕亮度”,请模仿"“由于您现在电脑当前亮度为X,已调至Y”"进行回答:
<bot>: 由于您现在电脑当前亮度为80,已调至50


In [1]:
re

'ai_user=pN1vaNEM/JFkEod5DPoEha|2023-06-08T02:24:16.518Z; ARRAffinity=4b55dda02c61fa6825af1a96193ff027e26cd4d1081dd1ee2ed88d88d93d93e9; ARRAffinitySameSite=4b55dda02c61fa6825af1a96193ff027e26cd4d1081dd1ee2ed88d88d93d93e9; .AspNetCore.Session=CfDJ8O%2ByR7SqPqxFlVz8eFNh6JFGsWDGqDT7Lzpzs4ImYU0jrnMFe6eWHIsbE52VYc8FRZV37J6ndDaxwjSxbvzE1fmeq53Mql%2B14DTKglIfKe9XpbajQ9UhJNbNo0v%2FYuluHZ5PoiEKJfG47olCOAp9n9QgGSclIv1Iak3PMKe9BclD; .AspNetCore.CookiesSSWeb=CfDJ8O-yR7SqPqxFlVz8eFNh6JEUTNyTeMBuOFEOcOPMwTcO-l0vrOFk0N9BKHtm_bRVY3W0u88P3I8fkCISyX_O7u15qf6PwF_D61PzBTgXJqHfBxQyHjf6rKXSu6YOduUWvsZG0qc4GNtZ7Jerm9yDkshdbEorp2nLmRpQ8obd91phvpKuqUxVl0avaXLOsBH8tOYY4FU7Zx6FtyNflVe6_-IkXRszZuU-9RLtwLSoKfMPjwMkfpUzUHECpB_puDrx4Hm6a6prhGkGKWifGk68HUZVUA9STh43cJklDVNwfwl6eBvUp8joU8TkaGoF1SHqN63HuOSpYv5gyjjS4Vd3Nls; ai_session=k434v/YYgul1PRSZB9GkaJ|1686191056621|1686192769861'

In [ ]:
input_text = """请你帮我根据以下instruction的要求提取出关键信息：
##instruction：要求格式为：
- 是否为电脑操作：
- 要求数字（未提及写未知）：
##input：
调整屏幕分辨率为1920*1080
##output:
"""
model.eval()
with torch.no_grad():
        
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to('cuda')
    
    out = model.generate(
        input_ids=input_ids,
        max_length=200,
        temperature=0.3,
        top_p = 0.95,
        # repetition_penalty = 1.15,
        # stopping_criteria = StoppingCriteriaList([stop_criteria])
        # do_sample = True
    )
    answer = tokenizer.decode(out[0])
    print(answer)
    # item['infer_answer'] = answer
    # print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')

In [68]:
input_text = """<用户>：请帮我打开微信
<ChatGLM-6B>:"""
model.eval()
with torch.no_grad():
        
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to('cuda')
    
    out = model.generate(
        input_ids=input_ids,
        max_length=200,
        temperature=0.1,
        top_p = 0.95,
        # repetition_penalty = 1.15,
        # stopping_criteria = StoppingCriteriaList([stop_criteria])
        # do_sample = True
    )
    answer = tokenizer.decode(out[0])
    print(answer)
    # item['infer_answer'] = answer
    # print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')

<用户>:请帮我打开微信
<ChatGLM-6B>: 很抱歉,作为一个语言模型,我无法直接打开微信应用程序。但是,您可以使用您的设备上的微信应用程序图标来打开它。如果该图标没有打开,您可以尝试在设备的设置中找到微信并打开它。


In [69]:
import json

if os.path.isfile('./operation/exe_location.json'):
    hash_map = json.loads('./operation/exe_location.json')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [79]:
hash_map = {
    'QQMusic.exe':'C:/Program Files (x86)\Tencent\QQMusic',
    'msedge.exe':'C:/Program Files (x86)\Microsoft\Edge\Application'
    }


In [80]:
with open('./operation/exe_location.json', 'w') as write_f:
	json.dump(hash_map, write_f, indent=4, ensure_ascii=False)


In [81]:
with open('./operation/exe_location.json','r') as load_f:
    hash_map = json.load(load_f)
    print(hash_map)
    print(type(load_dict))


{'QQMusic.exe': 'C:/Program Files (x86)\\Tencent\\QQMusic', 'msedge.exe': 'C:/Program Files (x86)\\Microsoft\\Edge\\Application'}
<class 'dict'>


In [5]:
import screen_brightness_control as sbc
sbc.get_brightness()[0]

10

In [32]:
sbc.get_brightness()[0]

100

In [6]:
sbc.set_brightness(100)

In [7]:
model.device

device(type='cuda', index=0)

In [1]:
from peft import PeftModel,LoraConfig,get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, AutoConfig,AutoModel


model = AutoModel.from_pretrained("THUDM/chatglm-6b-int4", load_in_8bit=True, trust_remote_code=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b-int4", trust_remote_code=True)

model.supports_gradient_checkpointing = True
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
# model.lm_head = CastOutputToFloat(model.lm_head)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
peft_config = LoraConfig(
    task_type='CAUSAL_LM', inference_mode=True,
    r=8,
    lora_alpha=32, lora_dropout=0.1,
)
# model = PeftModel.from_pretrained(model, "./tzh_model/medical_glm")
model = get_peft_model(model, peft_config)
model.is_parallelizable = True
model.model_parallel = True



c:\Python_tool\Anaconda\envs\torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin c:\Python_tool\Anaconda\envs\torch\lib\site-packages\bitsandbytes\libbitsandbytes_cuda116.dll c:\Python_tool\Anaconda\envs\torch\lib\site-packages\bitsandbytes libbitsandbytes_cuda116.dll
function 'cadam32bit_grad_fp32' not found
CUDA SETUP: Loading binary c:\Python_tool\Anaconda\envs\torch\lib\site-packages\bitsandbytes\libbitsandbytes_cuda116.dll...


c:\Python_tool\Anaconda\envs\torch\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like THUDM/chatglm-6b-int4 is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [1]:
from operation.operation import Operation3
inps = {'inputs':input('Input: ')}
test = Operation3(**inps)

{'QQMusic.exe': 'C:/Program Files (x86)\\Tencent\\QQMusic'}


In [2]:
test.fit()

c:\Python_tool\Anaconda\envs\torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
usage: Matching Task [-h] [--document-corpus DOCUMENT_CORPUS]
                     [--new-embed NEW_EMBED] [--document-embed DOCUMENT_EMBED]
                     [--k K] [--device DEVICE] [--model-name MODEL_NAME]
                     [--index_location INDEX_LOCATION]
Matching Task: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"6374c16d-bbe1-4643-92c1-507310150e28" --shell=9002 --transport="tcp" --iopub=9004 --f=c:\Users\Opti7080\AppData\Roaming\jupyter\runtime\kernel-v2-21952m58WaRixzUOk.json


SystemExit: 2

c:\Python_tool\Anaconda\envs\torch\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import bitsandbytes

In [17]:
!ls

01.wav
02.wav
03.wav
Untitled-1.ipynb
__pycache__
data
exe_location.json
generate_summary.py
inference.py
langchain-ChatGLM
main.py
map.py
operation
operation_idx.csv
play.py
recognition.py
record.py
renew_corpus.py
requirements.txt
result.wav
search_doc.py
search_doc_faiss.py
search_doc_hf.py
stop_criterion.py
synthesis.py
test_langchain.ipynb
test_thred.py
text2vec.ipynb
zh.wav


In [3]:
import torch
torch.__version__

'2.0.1+cu118'

In [5]:
import torch
inputs = tokenizer(queries, padding=True, truncation=True, return_tensors="pt")
with torch.no_grad():
    embeddings_que = model(**inputs)
    embeddings_ans = model(**tokenizer(text_right, padding=True, truncation=True, return_tensors="pt"))

In [6]:
embeddings_que.last_hidden_state.shape,embeddings_que.pooler_output.shape

(torch.Size([4, 22, 1024]), torch.Size([4, 1024]))

In [7]:
embeddings_que.last_hidden_state[:,0].numpy()

array([[-0.19582476, -0.22542259,  0.5878763 , ...,  0.51889557,
        -0.97035754,  0.72239023],
       [-0.36700755,  0.29916915,  0.31173083, ...,  1.2112145 ,
         0.05619805,  0.93658125],
       [-0.9832982 , -0.7832372 , -1.5102569 , ...,  1.4223759 ,
        -1.4264588 ,  0.75840706],
       [-1.4391133 ,  0.4106356 , -0.12989327, ..., -0.03974489,
        -1.0173508 , -0.79071593]], dtype=float32)

In [1]:

import faiss

In [8]:
from transformers import AutoModel,AutoTokenizer
import torch
device = 'cuda:0'
tokenizer = AutoTokenizer.from_pretrained("GanymedeNil/text2vec-large-chinese")
model = AutoModel.from_pretrained("GanymedeNil/text2vec-large-chinese").to(device)

def l2_normalization(data):
    if data.ndim == 1:
        return data / np.linalg.norm(data).reshape(-1,1)
    else:
        return data/np.linalg.norm(data,axis=1).reshape(-1,1)
def cls_pooling(model_output,return_tensors=False):
    if not return_tensors:
        return l2_normalization(model_output.last_hidden_state[:,0].cpu().numpy())
    else:
        return l2_normalization(model_output.last_hidden_state[:,0])
    


c:\Python_tool\Anaconda\envs\torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
corpus = []
with open('../data/document_corpus.txt','r',encoding='utf-8') as f:
    for line in f:
        corpus.append(line.strip('\n'))
def mean_pooling(model_output, attention_mask, return_tensors=False):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    output = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    if not return_tensors:
        return output.cpu().numpy()
    else:
        return output

In [27]:

import numpy as np
model.eval()
input_data = tokenizer(corpus, padding=True, return_tensors="pt")
with torch.no_grad():
    corpus_embeddings = model(**input_data.to(device))
# corpus_embeddings = cls_pooling(corpus_embeddings)
corpus_embeddings = mean_pooling(corpus_embeddings,attention_mask=input_data['attention_mask'])
corpus_embeddings = l2_normalization(corpus_embeddings)

In [28]:
query =  '帮我记录一下明天上午十点要买菜'
input_data = tokenizer(query, padding=True, return_tensors="pt")  
model.eval()     
with torch.no_grad():
    query_embeddings = model(**input_data.to('cuda'))
    # query_embeddings = cls_pooling(query_embeddings)
    query_embeddings = mean_pooling(query_embeddings,attention_mask=input_data['attention_mask'])
    feature_search = l2_normalization(query_embeddings)  

In [29]:
corpus_embeddings@feature_search.T

array([[0.23497748],
       [0.2247234 ],
       [0.27397212],
       [0.2500224 ]], dtype=float32)

In [30]:
corpus

['调节屏幕亮度', '询问显存占用', '进行事件提醒', '打开某个软件']

In [8]:
import faiss
import numpy as np
features = l2_normalization(np.load("./data/document_embed.npy"))
dim = features.shape[1]
index_ip = faiss.IndexFlatIP(dim)
index_ip.add(features)
print(index_ip.ntotal) 

6


In [99]:
np.array((corpus))

array(['调节屏幕亮度', '调整屏幕分辨率', '打开软件', '询问显存', '询问GPU占用', '创建记事本备忘录'],
      dtype='<U8')

In [80]:
aux_index = faiss.IndexIDMap(index_ip) 

RuntimeError: Error in __cdecl faiss::IndexIDMapTemplate<struct faiss::Index>::IndexIDMapTemplate(struct faiss::Index *) at D:\a\faiss-wheels\faiss-wheels\faiss\faiss\IndexIDMap.cpp:32: Error: 'index->ntotal == 0' failed: index must be empty on input

In [42]:
from search_doc_faiss import faiss_corpus
torch.cuda.empty_cache()
corpus = faiss_corpus()
corpus.fit()

[INFO]Loading Model
[INFO]Creating Document Embeddings
['调节屏幕亮度', '询问显存占用', '进行事件提醒', '打开软件', '询问GPU占用']
[INFO]Finishing Saving Indexing


In [4]:
corpus.corpus

['调节屏幕亮度', '询问显存占用', '创建备忘录进行事项提醒', '打开软件', '询问GPU占用']

In [20]:
index.reconstruct(0),index_ip.reconstruct(0)

(array([-0.01932951,  0.0241784 ,  0.02330983, ...,  0.02350627,
        -0.03408533,  0.0198594 ], dtype=float32),
 array([-0.04149882,  0.04571409, -0.02394647, ..., -0.03245361,
         0.04087428, -0.01606109], dtype=float32))

In [77]:
l2_normalization_1(features[1,:]),l2_normalization(features[1,:])

(array([[-0.03973337,  0.00838783,  0.01816207, ..., -0.00785893,
         -0.05274578,  0.00404025]], dtype=float32),
 array([-0.03973337,  0.00838783,  0.01816207, ..., -0.00785893,
        -0.05274578,  0.00404025], dtype=float32))

In [15]:
features.shape

(6, 1024)

In [12]:
import faiss
import numpy as np

query = '提醒我明天十点抢电影票'
index = faiss.read_index('./data/test.index')
with torch.no_grad():
    query_embeddings = model(**tokenizer(query, padding=True, return_tensors="pt").to(device))
query_embeddings = cls_pooling(query_embeddings)
feature_search = query_embeddings
# 检索最相似的topK个特征
topK = 5
D, I = index.search(feature_search, topK)
# 返回的D表示相似度（或者距离）, I表示检索的topK个特征id（索引）


In [52]:
query = '帮我记录一下明天十点提醒帮我买菜'
corpus_1 = """进行事件提醒"""
corpus_2 = """调节屏幕亮度"""

with torch.no_grad():
    query_embeddings = model(**tokenizer(query, padding=True, return_tensors="pt").to(device))
    feature_search = model(**tokenizer(corpus_1, padding=True, return_tensors="pt").to(device))
    feature_search_2 = model(**tokenizer(corpus_2, padding=True, return_tensors="pt").to(device))
feature_search = cls_pooling(feature_search)
query_embeddings = cls_pooling(query_embeddings)
feature_search_2 = cls_pooling(feature_search_2)

In [53]:
query_embeddings@feature_search.T,query_embeddings@feature_search_2.T

(array([[0.42430413]], dtype=float32), array([[0.3600703]], dtype=float32))

In [17]:
query_embeddings.shape,feature_search.shape

((1, 1024), (1, 1024))

In [7]:
D, I

(array([[0.38714254, 0.37287977, 0.3670662 , 0.3359719 , 0.30211502]],
       dtype=float32),
 array([[0, 2, 3, 1, 4]], dtype=int64))

In [8]:
corpus.corpus

['调节屏幕亮度', '询问显存占用', '创建备忘录进行事项提醒', '打开软件', '询问GPU占用']

In [27]:
D, I

(array([[0.38714254, 0.3670662 , 0.36462313, 0.3359719 , 0.30211502]],
       dtype=float32),
 array([[0, 3, 2, 1, 4]], dtype=int64))

In [28]:
# 保存索引
faiss.write_index(index_ip, './data/test.index')

# 读取索引
index = faiss.read_index('./data/test.index')


NameError: name 'index_ip' is not defined

In [22]:
import torch
query = '提醒我明天十点抢电影票'
with torch.no_grad():
    query_embeddings = model(**tokenizer(query, padding=True, return_tensors="pt").to('cuda'))
query_embeddings = cls_pooling(query_embeddings)
feature_search = query_embeddings
# 检索最相似的topK个特征
topK = 5
D, I = index.search(feature_search, topK)
# 返回的D表示相似度（或者距离）, I表示检索的topK个特征id（索引）

NameError: name 'cls_pooling' is not defined

In [47]:
index

<faiss.swigfaiss.IndexFlat; proxy of <Swig Object of type 'faiss::IndexFlat *' at 0x000001A3B59EF900> >

In [48]:
D, I

(array([[0.7988719 , 0.56085855, 0.4258615 , 0.41730604, 0.3314243 ]],
       dtype=float32),
 array([[1, 5, 3, 0, 2]], dtype=int64))

In [49]:
corpus.corpus

['调节屏幕亮度', '询问显存占用', '创建记事本备忘录', '调整屏幕分辨率', '打开软件', '询问GPU占用']

In [34]:
def normalization(data):
    _range = np.max(data) - np.min(data)
    return (data - np.min(data)) / _range
 
 
def standardization(data):
    mu = np.mean(data, axis=1)
    sigma = np.std(data, axis=1)
    return (data - mu) / sigma



np.sum(l2_normalization(query_embeddings))

-0.38731813

In [28]:
query_embeddings.shape

(1, 1024)

In [35]:
np.linalg.norm(l2_normalization(query_embeddings))

1.0

In [21]:
query_embeddings

array([[-0.19582501, -0.2254237 ,  0.5878755 , ...,  0.51889545,
        -0.9703573 ,  0.7223894 ]], dtype=float32)

In [20]:
D,I

(array([[835.5797 , 564.0149 , 397.68317, 339.4082 , 246.25342]],
       dtype=float32),
 array([[0, 1, 4, 3, 2]], dtype=int64))

In [ ]:
index_ids = faiss.IndexFlatIP(2048)
index_ids = faiss.IndexIDMap(index_ids)

# 添加特征，并指定id，注意添加的id类型为int64
ids = 20
feature_ids = np.random.random((1, 2048)).astype('float32')
index_ids.add_with_ids(feature_ids, np.array((ids,)).astype('int64'))


In [4]:
index_ip

<faiss.swigfaiss.IndexFlatIP; proxy of <Swig Object of type 'faiss::IndexFlatIP *' at 0x000001D6C6EFF570> >

In [8]:
from text2vec import cos_sim
cos_similarity = cos_sim(embeddings_que.last_hidden_state[:,0],embeddings_ans.last_hidden_state[:,0])
print(cos_similarity)

tensor([[0.9465, 0.6756, 0.2798, 0.3837, 0.4044, 0.2968, 0.2742, 0.3308, 0.3206,
         0.4471, 0.4081],
        [0.3055, 0.2763, 0.7725, 0.6567, 0.3291, 0.7498, 0.2632, 0.4862, 0.5845,
         0.4109, 0.4650],
        [0.3777, 0.3504, 0.3729, 0.5051, 0.5157, 0.4793, 0.5053, 0.4642, 0.5326,
         0.6118, 0.5007],
        [0.3411, 0.3380, 0.3204, 0.4363, 0.4000, 0.4164, 0.3400, 0.4188, 0.5530,
         0.3807, 0.5776]])


In [41]:
cos_similarity.argmax(dim=1)

tensor([ 0,  2,  9, 10])

In [13]:
tensor_cpu.argmax(dim=1)

tensor([ 0,  2,  9, 10])

In [22]:
from text2vec import cos_sim

In [14]:
tensor_cpu.shape

torch.Size([4, 11])

In [15]:
tensor_cpu

tensor([[0.9719, 0.8737, 0.7577, 0.7623, 0.7647, 0.7870, 0.7640, 0.7554, 0.7812,
         0.7794, 0.7791],
        [0.7596, 0.8073, 0.9713, 0.9700, 0.8659, 0.9233, 0.8840, 0.8633, 0.9166,
         0.8967, 0.9084],
        [0.7571, 0.7907, 0.8844, 0.8987, 0.8838, 0.9239, 0.9227, 0.8731, 0.9319,
         0.9432, 0.9264],
        [0.7539, 0.7718, 0.8788, 0.9024, 0.8693, 0.9033, 0.8655, 0.8638, 0.9149,
         0.8890, 0.9299]])

In [23]:
import screen_brightness_control as sbc

In [29]:
sbc.get_brightness()

[100]

In [35]:
sbc.set_brightness(90)

In [37]:
import os
os.system('nvidia-smi')

0

In [38]:
import pynvml
pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
info = pynvml.nvmlDeviceGetMemoryInfo(handle)
print(f"Total memory: {info.total/1024**2} MB")
print(f"Free memory: {info.free/1024**2} MB")
print(f"Used memory: {info.used/1024**2} MB")

Total memory: 8192.0 MB
Free memory: 6604.09375 MB
Used memory: 1587.90625 MB


In [41]:
# from operation.screen_brightness import operation
# opt = operation(50)
# opt.fit()
from operation.display_graphic import operation
opt = operation()
opt.fit()

Total memory: 8192.00 MB
Free memory: 6604.09 MB
Used memory: 1587.91 MB


In [1]:
from operation import operation



In [2]:
selected_idx = 0
inp = {"brightness":100}
opt = eval(f"operation.Operation{selected_idx}")(**inp)

In [3]:
opt.fit()

'已调至100%'

In [43]:
def time_suffix():
    return time.strftime("%Y%m%d%H%M%S", time.localtime(time.time()))

In [45]:
import time 
time_suffix()

'20230529144958'

In [47]:
os.path.join('LenoMate',time_suffix(),'.txt')

'LenoMate\\20230529145105\\.txt'

In [48]:
summary='love'

In [51]:
'LenoMate_'+summary+'_'+time_suffix()+'.txt'

'LenoMate_love_20230529145339.txt'

In [55]:
def test_args_kwargs(arg1, arg2, arg3,arg4=5):
    print("arg1:", arg1)
    print("arg2:", arg2)
    print("arg3:", arg3)

In [56]:
kwargs = {"arg3": 3, "arg2": "two", "arg1": 5}
test_args_kwargs(**kwargs)

#result
arg1: 5
arg2: 'two'
arg3: 3

arg1: 5
arg2: two
arg3: 3


In [61]:
import os
import time
default_path = os.path.join(os.path.expanduser("~"), "Desktop")
class operation():
    def __init__(self,inputs,path=default_path,summary='notebook'):
        self.path = path
        self.inputs = inputs
        self.summary = summary
    def fit(self):
        file_name = 'LenoMate_'+self.summary+'_'+time_suffix()+'.txt'
        file_name = os.path.join(default_path,file_name)
        with open(file_name,'w',encoding='utf-8') as f:
            f.write(self.inputs)
        f.close()
inp = {'inputs':'s','summary':'love'}
tst = operation(**inp)

In [62]:
tst.summary

'love'

In [57]:
import re

string=""" instruction:
- 任务: 调整屏幕亮度
- 要求数字: 10
output:
- 将屏幕亮度调低到10"""
re.findall(r"\d+\.?\d*",string)

['10', '10']

In [1]:
import threading
import queue
import numpy as np

# 模拟加载和运行模型的函数
def load_and_run_model(input_queue):
    # 模型加载过程，可以根据实际情况进行编写
    print("模型加载中...")
    # 模型加载完成
    print("模型加载完成")

    while True:
        try:
            # 从输入队列中获取输入数据
            input_data = input_queue.get()
            # 在这里进行模型处理
            # 模型处理逻辑...
            # 将输入数据转换为numpy数组
            input_array = eval(input_data)
            # 运行模型，这里我们简单地将输入数据乘以2
            output_array = input_array * 2
            # 将结果转换为字符串
            output_data = str(output_array)

            # 处理完成后，可以将结果返回给主线程，或者进行其他操作
            print("模型输出：", output_data)
        except KeyboardInterrupt:
            break

# 创建一个输入队列
input_queue = queue.Queue()

# 创建一个线程，用于加载和运行模型
model_thread = threading.Thread(target=load_and_run_model, args=(input_queue,))
# 设置线程为后台线程，使程序可以退出
model_thread.daemon = True
# 启动线程
model_thread.start()

while True:
    # 接收用户输入
    input_data = input("请输入数据（输入'exit'退出）：")
    if input_data == "exit":
        break
    # 将输入数据放入输入队列
    input_queue.put(input_data)

# 退出程序时，清空输入队列并等待模型线程结束
input_queue.join()


模型加载中...
模型加载完成
模型输出： 2
模型输出： 4
模型输出： 6
模型输出： 8
模型输出： 10
模型输出： 12


Exception in thread Thread-3 (load_and_run_model):
Traceback (most recent call last):
  File "c:\Python_tool\Anaconda\envs\torch\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Python_tool\Anaconda\envs\torch\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Opti7080\AppData\Local\Temp\ipykernel_8504\1912693514.py", line 19, in load_and_run_model
  File "<string>", line 0
    
SyntaxError: invalid syntax


In [17]:
import re
pattern =  "数字(?:\(.+?\))?:\s*"
text = '要求数字(没有则写不含):1111'
re.split(pattern,text),re.findall(pattern,text)

(['要求', '1111'], ['数字:'])

In [5]:
import re
answer = """
数字:未知
"""
try:
    num = int(re.findall(r"\d+\.?\d*",answer.split('数字:')[1])[0])
except:
    num = answer.split('数字:')[1]

In [3]:
import ctypes
import tkinter as tk
from tkinter import ttk
from pycaw.pycaw import AudioUtilities, ISimpleAudioVolume

# 创建一个Tkinter窗口
window = tk.Tk()
window.title("音量控制")
window.geometry("300x100")

# 获取默认的音频会话管理器
sessions = AudioUtilities.GetAllSessions()
for session in sessions:
    volume = session._ctl.QueryInterface(ISimpleAudioVolume)
    if session.Process and session.Process.name() == "python.exe":
        volume.SetMasterVolume(0.5, None)  # 初始化音量为50%

# 函数：调节音量
def set_volume(vol):
    for session in sessions:
        volume = session._ctl.QueryInterface(ISimpleAudioVolume)
        if session.Process and session.Process.name() == "python.exe":
            volume.SetMasterVolume(vol / 100, None)

# 创建一个音量控制滑块
volume_slider = ttk.Scale(window, from_=0, to=100, orient="horizontal", command=set_volume)
volume_slider.set(50)
volume_slider.pack(pady=20)

# 运行GUI主循环
window.mainloop()


KeyboardInterrupt: 

: 

In [10]:
import os
import re
sorted([f for f in os.listdir('../') if re.match('pytorch_model-(\d+)-of-(\d+).bin',f)])

[]

In [8]:
os.listdir('../')

['.git',
 '.gitignore',
 '.idea',
 'audio',
 'data',
 'lenochat.py',
 'main.py',
 'maintext.py',
 'operation',
 'QA.html',
 'README.md',
 'requirements.txt',
 'stop_criterion.py',
 'test_file',
 'utils',
 'web.py',
 '__pycache__']

In [1]:


from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

# 获取音量值，0.0代表最大，-65.25代表最小
vl = volume.GetMasterVolumeLevel()
print('get',vl)


# 设置静音，mute为1代表是静音，为0代表不是静音
volume.SetMute(0, None)
volume.SetMasterVolumeLevel(-65.25 + 8.26, None)
vl = volume.GetMasterVolumeLevel()
print('get',vl)
# 设置音量大小为60%
# volume.SetMasterVolumeLevel(-7.63, None)

get -10.814563751220703
get -56.9900016784668


In [4]:


from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
vl = volume.GetMasterVolumeLevel()
print('get',vl)

get -9.064839363098145


In [19]:
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

# 获取音量值，0.0代表最大，-65.25代表最小
vl = volume.GetMasterVolumeLevel()
print('get',vl)


# 设置静音，mute为1代表是静音，为0代表不是静音
volume.SetMute(0, None)
volume.SetMasterVolumeLevel(-8.65, None)

get -8.649999618530273


0

In [20]:
volume.GetMasterVolumeLevel()

-8.649999618530273

In [57]:
from transformers import AutoModel,AutoTokenizer
import torch
import numpy as np

model_sim =  AutoModel.from_pretrained("GanymedeNil/text2vec-large-chinese").to('cuda')
tokenizer_sim = AutoTokenizer.from_pretrained("GanymedeNil/text2vec-large-chinese")

class vol_ctrl:
    def __init__(self):
        
        """初始化音量对应"""   
        # 音量对应关系
        # 0% --- -65.25
        # 55% --- -8.92
        # 100% --- 0.0
        self.volume_match={0: -65.25, 1: -56.99, 2: -51.67, 3: -47.74, 4: -44.62, 5: -42.03, 6: -39.81, 7: -37.89, 8: -36.17, 9: -34.63, 10: -33.24,
            11: -31.96, 12: -30.78, 13: -29.68, 14: -28.66, 15: -27.7, 16: -26.8, 17: -25.95, 18: -25.15, 19: -24.38, 20: -23.65,
            21: -22.96, 22: -22.3, 23: -21.66, 24: -21.05, 25: -20.46, 26: -19.9, 27: -19.35, 28: -18.82, 29: -18.32, 30: -17.82,
            31: -17.35, 32: -16.88, 33: -16.44, 34: -16.0, 35: -15.58, 36: -15.16, 37: -14.76, 38: -14.37, 39: -13.99, 40: -13.62,
            41: -13.26, 42: -12.9, 43: -12.56, 44: -12.22, 45: -11.89, 46: -11.56, 47: -11.24, 48: -10.93, 49: -10.63, 50: -10.33,
            51: -10.04, 52: -9.75, 53: -9.47, 54: -9.19, 55: -8.92, 56: -8.65, 57: -8.39, 58: -8.13, 59: -7.88, 60: -7.63,
            61: -7.38, 62: -7.14, 63: -6.9, 64: -6.67, 65: -6.44, 66: -6.21, 67: -5.99, 68: -5.76, 69: -5.55, 70: -5.33,
            71: -5.12, 72: -4.91, 73: -4.71, 74: -4.5, 75: -4.3, 76: -4.11, 77: -3.91, 78: -3.72, 79: -3.53, 80: -3.34,
            81: -3.15, 82: -2.97, 83: -2.79, 84: -2.61, 85: -2.43, 86: -2.26, 87: -2.09, 88: -1.91, 89: -1.75, 90: -1.58,
            91: -1.41, 92: -1.25, 93: -1.09, 94: -0.93, 95: -0.77, 96: -0.61, 97: -0.46, 98: -0.3, 99: -0.15, 100: 0.0}
        #音量值反对应
        self.volume_match_2={-65.25:0, -56.99:1, -51.67:2, -47.74:3, -44.62:4, -42.03:5, -39.81:6,-37.89:7, -36.17:8, -34.63:9, -33.24:10,
            -31.96:11, -30.78:12, -29.68:13, -28.66:14, -27.7:15, -26.8:16, -25.95:17, -25.15:18,  -24.38:19, -23.65:20,
            -22.96:21, -22.3:22, -21.66:23, -21.05:24, -20.46:25, -19.9:26, -19.35:27,  -18.82:28,  -18.32:29,  -17.82:30,
            -17.35:31, -16.88:32, -16.44:33, -16.0:34, -15.58:35,-15.16:36, -14.76:37, -14.37:38,  -13.99:39, -13.62:40,
            -13.26:41, -12.9:42, -12.56:43, -12.22:44,-11.89:45, -11.56:46, -11.24:47, -10.93:48,  -10.63:49, -10.33:50,
            -10.04:51, -9.75:52, -9.47:53, -9.19:54, -8.92:55, -8.65:56, -8.39:57, -8.13:58,  -7.88:59,  -7.63:60,
            -7.38:61, -7.14:62, -6.9:63, -6.67:64, -6.44:65, -6.21:66, -5.99:67,  -5.76:68,  -5.55:69, -5.33:70,
            -5.12:71,-4.91:72, -4.71:73, -4.5:74, -4.3:75, -4.11:76,  -3.91:77, -3.72:78,  -3.53:79,  -3.34:80,
            -3.15:81, -2.97:82, -2.79:83, -2.61:84, -2.43:85, -2.26:86, -2.09:87,  -1.91:88, -1.75:89, -1.58:90,
           -1.41:91, -1.25:92, -1.09:93, -0.93:94, -0.77:95, -0.61:96, -0.46:97, -0.3:98,  -0.15:99,  0.0:100}
        self.devices = AudioUtilities.GetSpeakers()
        self.interface = self.devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
        self.volume = cast(self.interface, POINTER(IAudioEndpointVolume))

    def mute_all(self,mute=True):
        self.mute = mute
        if self.mute: self.volume.SetMute(1, None)
        else: self.volume.SetMute(0, None)

    def alter(self,num):
        if num > 0:
            self.mute_all(mute = False)
        vl = self.volume.GetMasterVolumeLevel()
        print('get current volumn',vl)
        self.volume.SetMasterVolumeLevel(self.volume_match[int(num)], None)
        print('alter to ',num)

class Operation5():
    def __init__(self,inputs,model_sim,tokenizer_sim) -> None:
        
        self.vol_ctrl = vol_ctrl()
        self.inputs = inputs
        self.model_sim = model_sim
        self.tokenizer_sim = tokenizer_sim

    def fit(self,model,tokenizer):
        self.judge()
        if not self.selected: self.vol_ctrl.mute_all()
        pass

    def judge(self):
        _mute = '静音，取消静音'
        _normal = '调节音量'
        corpus = [_mute,_normal]
        self.model_sim.eval()
        input_data = self.tokenizer_sim(self.inputs, return_tensors="pt")
        corp = self.tokenizer_sim(corpus, return_tensors="pt",padding=True)
        with torch.no_grad():
            corpus_embeddings = self.model_sim(**corp.to('cuda'))
            input_embeddings = self.model_sim(**input_data.to('cuda'))
        corpus_embeddings = self._pooling(corpus_embeddings,attention_mask=corp['attention_mask'])
        input_embeddings = self._pooling(input_embeddings,attention_mask=input_data['attention_mask'])
        self.selected = corpus[(corpus_embeddings@input_embeddings.T).argmax(axis=0)[0]]
        print(f"[音量调节功能]音量功能匹配为'{self.selected}'")

    def _pooling(self,model_output,attention_mask):
        token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        output = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return self._l2_normalization(output.cpu().numpy())
    
    def _l2_normalization(self,data):
        if data.ndim == 1:
            return data / np.linalg.norm(data).reshape(-1,1)
        else:
            return data/ np.linalg.norm(data,axis=1).reshape(-1,1)


In [4]:
import os
import openai
openai.api_type = "azure"
openai.api_base = "https://openaipoc-kepler.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = '14639a3cad0f4ba38483194f82c6920b'

messageshistory = [
                    {"role":"system","content":"You are an AI assistant that helps people find information."},
                    {"role":"user","content":"请给我介绍一下联想企业"},
                    ]
response = openai.ChatCompletion.create(
      engine="Kepler1",
      messages = messageshistory,
      temperature=0.5,
      max_tokens=300,
      top_p=0.95,
      frequency_penalty=0,
      presence_penalty=0,
      stop=None)
print('AI bot:',response['choices'][0]['message']['content'])
bot = response['choices'][0]['message']['content']

AI bot: 联想集团（Lenovo Group Limited）是一家总部位于中国北京的跨国科技公司，成立于1984年。它在全球范围内提供消费类电子产品，如个人电脑、笔记本电脑、平板电脑、智能手机、服务器、工作站、网络存储设备和智能电视等。联想是全球最大的个人电脑制造商之一，同时也是全球最大的智能手机供应商之一。

联想的创始人兼董事局主席是柳传志，他是中国著名的企业家之一。在过去的几十年里，联想通过不断创新和扩大市场份额，逐渐成为全球科技产业的领导者。2005年，联想收购了IBM的个人电脑业务，使其在全球个人电脑市场中占据了重要地位。此次收购使联想的品牌知名度大大提高，同时也为公司在全球市场的发展奠定了基础。

近年来，联想还不断扩大其业务领域，涉足包括人工智能、大数据、云计算和物联网等新兴科技领域。联想致力于为客户提供高质量、高性能的产品和解决方案，满足不同客户的需求。

总的来说，联想集团是一家具有全球影响力的科技公司，为全球数以亿计的用户提供了优质的科技产品和服务。


In [9]:
# -*- coding: utf-8 -*-
'''
@name: 系统信息 / SystemInfo
@author: PurePeace
@time: 2020年8月17日
@version: 0.1
'''

from typing import List, Dict, Any

import os
import time
import psutil
import platform
import hashlib
import re
import sys


from cachelib import SimpleCache
cache = SimpleCache()


UNIX: bool = os.name == 'posix'
SYS: str = platform.system()


class CpuConstants:
    def __init__(self):
        '''
        初始化CPU常量（多平台）

        Returns
        -------
        self.

        '''
        self.WMI = None
        self.initialed: bool = False
        self.cpuList: list = [] # windows only

        self.cpuCount: int = 0 # 物理cpu数量
        self.cpuCore: int = 0 # cpu物理核心数
        self.cpuThreads: int = 0 # cpu逻辑核心数
        self.cpuName: str = '' # cpu型号

        self.Update(True)


    def Update(self, update: bool = False) -> None:
        '''
        更新cpu数据

        Returns
        -------
        None.

        '''
        if UNIX: self.GetCpuConstantsUnix(update)
        else: self.GetCpuConstantsWindows(update)

        self.initialed: bool = True


    @property
    def getDict(self) -> Dict[int, str]:
        '''
        以字典格式获取当前cpu常量

        Returns
        -------
        Dict[int, str]
            DESCRIPTION.

        '''
        if not self.initialed: self.Update()
        return {
            'cpu_count': self.cpuCount,
            'cpu_name': self.cpuName,
            'cpu_core': self.cpuCore,
            'cpu_threads': self.cpuThreads
        }


    def GetCpuConstantsUnix(self, update: bool = False) -> None:
        '''
        获取unix下的cpu信息

        Parameters
        ----------
        update : bool, optional
            DESCRIPTION. The default is False.

        Returns
        -------
        None
            DESCRIPTION.

        '''
        if update or not self.initialed:
            ids: list = re.findall("physical id.+", readFile('/proc/cpuinfo'))

            # 物理cpu个数
            self.cpuCount: int = len(set(ids))

            # cpu型号（名称）
            self.cpuName: str = self.getCpuTypeUnix()


            self.GetCpuConstantsBoth()


    def InitWmi(self) -> None:
        '''
        初始化wmi（for windows）

        Returns
        -------
        None
            DESCRIPTION.

        '''
        import wmi
        self.WMI = wmi.WMI()


    def GetCpuConstantsBoth(self, update: bool = False) -> None:
        '''
        获取多平台共用的cpu信息

        Parameters
        ----------
        update : bool, optional
            强制更新数据. The default is False.

        Returns
        -------
        None
            DESCRIPTION.

        '''
        if update or not self.initialed:

            # cpu逻辑核心数
            self.cpuThreads: int = psutil.cpu_count()

            # cpu物理核心数
            self.cpuCore: int = psutil.cpu_count(logical=False)


    def GetCpuConstantsWindows(self, update: bool = False) -> None:
        '''
        获取windows平台的cpu信息

        Parameters
        ----------
        update : bool, optional
            强制更新数据. The default is False.

        Returns
        -------
        None
            DESCRIPTION.

        '''
        if update or not self.initialed:

            # 初始化wmi
            if self.WMI == None: self.InitWmi()

            # cpu列表
            self.cpuList: list = self.WMI.Win32_Processor()

            # 物理cpu个数
            self.cpuCount: int = len(self.cpuList)

            # cpu型号（名称）
            self.cpuName: str = self.cpuList[0].Name


            self.GetCpuConstantsBoth()


    @staticmethod
    def getCpuTypeUnix() -> str:
        '''
        获取CPU型号（unix）

        Returns
        -------
        str
            CPU型号.

        '''
        cpuinfo: str = readFile('/proc/cpuinfo')
        rep: str = 'model\s+name\s+:\s+(.+)'
        tmp = re.search(rep,cpuinfo,re.I)
        cpuType: str = ''
        if tmp:
            cpuType: str = tmp.groups()[0]
        else:
            cpuinfo = ExecShellUnix('LANG="en_US.UTF-8" && lscpu')[0]
            rep = 'Model\s+name:\s+(.+)'
            tmp = re.search(rep,cpuinfo,re.I)
            if tmp: cpuType = tmp.groups()[0]
        return cpuType


def GetCpuInfo(interval: int = 1) -> Dict[str, Any]:
    '''
    获取CPU信息

    Parameters
    ----------
    interval : int, optional
        DESCRIPTION. The default is 1.

    Returns
    -------
    Dict[float, list, dict]
        DESCRIPTION.

    '''
    time.sleep(0.5)


    # cpu总使用率
    used: float = psutil.cpu_percent(interval)

    # 每个逻辑cpu使用率
    usedList: List[float] = psutil.cpu_percent(percpu=True)


    return {'used': used, 'used_list': usedList, **cpuConstants.getDict}


def readFile(filename: str) -> str:
    '''
    读取文件内容

    Parameters
    ----------
    filename : str
        文件名.

    Returns
    -------
    str
        文件内容.

    '''
    try:
        with open(filename, 'r', encoding='utf-8') as file: return file.read()
    except:
        pass

    return ''


def GetLoadAverage() -> dict:
    '''
    获取服务器负载状态（多平台）

    Returns
    -------
    dict
        DESCRIPTION.

    '''
    try: c: list = os.getloadavg()
    except: c: list = [0,0,0]
    data: dict = {i: c[idx] for idx, i in enumerate(('one', 'five', 'fifteen'))}
    data['max'] = psutil.cpu_count() * 2
    data['limit'] = data['max']
    data['safe'] = data['max'] * 0.75
    return data


def GetMemInfo() -> dict:
    '''
    获取内存信息（多平台）

    Returns
    -------
    dict
        DESCRIPTION.

    '''
    if UNIX: return GetMemInfoUnix()
    return GetMemInfoWindows()


def GetMemInfoUnix() -> Dict[str, int]:
    '''
    获取内存信息（unix）

    Returns
    -------
    dict
        DESCRIPTION.

    '''
    mem = psutil.virtual_memory()
    memInfo: dict = {
        'memTotal': ToSizeInt(mem.total, 'MB'),
        'memFree': ToSizeInt(mem.free, 'MB'),
        'memBuffers': ToSizeInt(mem.buffers, 'MB'),
        'memCached': ToSizeInt(mem.cached, 'MB'),
    }
    memInfo['memRealUsed'] = \
        memInfo['memTotal'] - \
        memInfo['memFree'] - \
        memInfo['memBuffers'] - \
        memInfo['memCached']

    memInfo['memUsedPercent'] = memInfo['memRealUsed'] / memInfo['memTotal'] * 100

    return memInfo


def GetMemInfoWindows() -> dict:
    '''
    获取内存信息（windows）

    Returns
    -------
    dict
        DESCRIPTION.

    '''
    mem = psutil.virtual_memory()
    memInfo: dict = {
        'memTotal': ToSizeInt(mem.total, 'MB'),
        'memFree': ToSizeInt(mem.free, 'MB'),
        'memRealUsed': ToSizeInt(mem.used, 'MB'),
        'menUsedPercent': mem.used / mem.total * 100
    }

    return memInfo


def ToSizeInt(byte: int, target: str) -> int:
    '''
    将字节大小转换为目标单位的大小

    Parameters
    ----------
    byte : int
        int格式的字节大小（bytes size）
    target : str
        目标单位，str.

    Returns
    -------
    int
        转换为目标单位后的字节大小.

    '''
    return int(byte/1024**(('KB','MB','GB','TB').index(target) + 1))


def ToSizeString(byte: int) -> str:
    '''
    获取字节大小字符串

    Parameters
    ----------
    byte : int
        int格式的字节大小（bytes size）.

    Returns
    -------
    str
        自动转换后的大小字符串，如：6.90 GB.

    '''
    units: tuple = ('b','KB','MB','GB','TB')
    re = lambda: '{:.2f} {}'.format(byte, u)
    for u in units:
        if byte < 1024: return re()
        byte /= 1024
    return re()


def GetDiskInfo() -> list:
    '''
    获取磁盘信息（多平台）

    Returns
    -------
    list
        列表.

    '''
    try:
        if UNIX: return GetDiskInfoUnix()
        return GetDiskInfoWindows()
    except Exception as err:
        print('获取磁盘信息异常（unix: {}）：'.format(UNIX), err)
        return []


def GetDiskInfoWindows() -> list:
    '''
    获取磁盘信息Windows

    Returns
    -------
    diskInfo : list
        列表.

    '''
    diskIo: list = psutil.disk_partitions()
    diskInfo: list = []
    for disk in diskIo:
        tmp: dict = {}
        try:
            tmp['path'] = disk.mountpoint.replace("\\","/")
            usage = psutil.disk_usage(disk.mountpoint)
            tmp['size'] = {
                'total': usage.total,
                'used': usage.used,
                'free': usage.free,
                'percent': usage.percent
            }
            tmp['fstype'] = disk.fstype
            tmp['inodes'] = False
            diskInfo.append(tmp)
        except:
            pass
    return diskInfo


def GetDiskInfoUnix() -> list:
     '''
    获取硬盘分区信息（unix）

    Returns
    -------
    list
        DESCRIPTION.

    '''
     temp: list = (
         ExecShellUnix("df -h -P|grep '/'|grep -v tmpfs")[0]).split('\n')
     tempInodes: list = (
         ExecShellUnix("df -i -P|grep '/'|grep -v tmpfs")[0]).split('\n')
     diskInfo: list = []
     n: int = 0
     cuts: list = [
         '/mnt/cdrom',
         '/boot',
         '/boot/efi',
         '/dev',
         '/dev/shm',
         '/run/lock',
         '/run',
         '/run/shm',
         '/run/user'
     ]
     for tmp in temp:
         n += 1
         try:
             inodes: list = tempInodes[n-1].split()
             disk: list = tmp.split()
             if len(disk) < 5: continue
             if disk[1].find('M') != -1: continue
             if disk[1].find('K') != -1: continue
             if len(disk[5].split('/')) > 10: continue
             if disk[5] in cuts: continue
             if disk[5].find('docker') != -1: continue
             arr = {}
             arr['path'] = disk[5]
             tmp1 = [disk[1],disk[2],disk[3],disk[4]]
             arr['size'] = tmp1
             arr['inodes'] = [inodes[1],inodes[2],inodes[3],inodes[4]]
             diskInfo.append(arr)
         except Exception as ex:
             print('信息获取错误：', str(ex))
             continue
     return diskInfo



def md5(strings: str) -> str:
    '''
    生成md5

    Parameters
    ----------
    strings : TYPE
        要进行hash处理的字符串

    Returns
    -------
    str[32]
        hash后的字符串.

    '''

    m = hashlib.md5()
    m.update(strings.encode('utf-8'))
    return m.hexdigest()


def GetErrorInfo() -> str:
    '''
    获取traceback中的错误

    Returns
    -------
    str
        DESCRIPTION.

    '''
    import traceback
    errorMsg = traceback.format_exc()
    return errorMsg


def ExecShellUnix(cmdstring: str, shell=True):
    '''
    执行Shell命令（Unix）

    Parameters
    ----------
    cmdstring : str
        DESCRIPTION.
    shell : TYPE, optional
        DESCRIPTION. The default is True.

    Returns
    -------
    a : TYPE
        DESCRIPTION.
    e : TYPE
        DESCRIPTION.

    '''
    a: str = ''
    e: str = ''
    import subprocess,tempfile

    try:
        rx: str = md5(cmdstring)
        succ_f = tempfile.SpooledTemporaryFile(
            max_size = 4096,
            mode = 'wb+',
            suffix = '_succ',
            prefix = 'btex_' + rx ,
            dir = '/dev/shm'
        )
        err_f = tempfile.SpooledTemporaryFile(
            max_size = 4096,
            mode = 'wb+',
            suffix = '_err',
            prefix = 'btex_' + rx ,
            dir = '/dev/shm'
        )
        sub = subprocess.Popen(
            cmdstring,
            close_fds = True,
            shell = shell,
            bufsize = 128,
            stdout = succ_f,
            stderr = err_f
        )
        sub.wait()
        err_f.seek(0)
        succ_f.seek(0)
        a = succ_f.read()
        e = err_f.read()
        if not err_f.closed: err_f.close()
        if not succ_f.closed: succ_f.close()
    except Exception as err:
        print(err)
    try:
        if type(a) == bytes: a = a.decode('utf-8')
        if type(e) == bytes: e = e.decode('utf-8')
    except Exception as err:
        print(err)

    return a,e


def GetNetWork() -> dict:
    '''
    获取系统网络信息

    Returns
    -------
    dict
        DESCRIPTION.

    '''
    networkIo: list = [0,0,0,0]
    cache_timeout: int = 86400
    try:
        networkIo = psutil.net_io_counters()[:4]
    except:
        pass

    otime = cache.get("otime")
    if not otime:
        otime = time.time()
        cache.set('up',networkIo[0],cache_timeout)
        cache.set('down',networkIo[1],cache_timeout)
        cache.set('otime',otime ,cache_timeout)

    ntime = time.time()
    networkInfo: dict = {'up': 0, 'down': 0}
    networkInfo['upTotal']   = networkIo[0]
    networkInfo['downTotal'] = networkIo[1]
    try:
        networkInfo['up'] = round(
            float(networkIo[0] - cache.get("up")) / 1024 / (ntime - otime),
            2
        )
        networkInfo['down'] = round(
            float(networkIo[1] - cache.get("down")) / 1024 / (ntime -  otime),
            2
        )
    except:
        pass

    networkInfo['downPackets'] = networkIo[3]
    networkInfo['upPackets'] = networkIo[2]

    cache.set('up',networkIo[0],cache_timeout)
    cache.set('down',networkIo[1],cache_timeout)
    cache.set('otime', time.time(),cache_timeout)

    return networkInfo


def GetSystemInfo() -> dict:
    systemInfo: dict = {}
    systemInfo['cpu'] = GetCpuInfo()
    systemInfo['load'] = GetLoadAverage()
    systemInfo['mem'] = GetMemInfo()
    systemInfo['disk'] = GetDiskInfo()

    return systemInfo



def GetIoReadWrite() -> Dict[str, int]:
    '''
    获取系统IO读写

    Returns
    -------
    dict
        DESCRIPTION.

    '''
    ioDisk = psutil.disk_io_counters()
    ioTotal: dict = {}
    ioTotal['write'] = GetIoWrite(ioDisk.write_bytes)
    ioTotal['read'] = GetIoRead(ioDisk.read_bytes)
    return ioTotal


def GetIoWrite(ioWrite: int) -> int:
    '''
    获取IO写

    Parameters
    ----------
    ioWrite : TYPE
        DESCRIPTION.

    Returns
    -------
    int
        DESCRIPTION.

    '''
    diskWrite: int = 0
    oldWrite: int = cache.get('io_write')
    if not oldWrite:
        cache.set('io_write', ioWrite)
        return diskWrite;

    oldTime: float = cache.get('io_time')
    newTime: float = time.time()
    if not oldTime: oldTime = newTime
    ioEnd: int = (ioWrite - oldWrite)
    timeEnd: float = (time.time() - oldTime)
    if ioEnd > 0:
        if timeEnd < 1: timeEnd = 1
        diskWrite = ioEnd / timeEnd
    cache.set('io_write',ioWrite)
    cache.set('io_time',newTime)
    if diskWrite > 0: return int(diskWrite)
    return 0


def GetIoRead(ioRead):
    '''
    读取IO读

    Parameters
    ----------
    ioRead : TYPE
        DESCRIPTION.

    Returns
    -------
    TYPE
        DESCRIPTION.

    '''
    diskRead: int = 0
    oldRead: int = cache.get('io_read')
    if not oldRead:
        cache.set('io_read',ioRead)
        return diskRead;
    oldTime: float = cache.get('io_time')
    newTime: float = time.time()
    if not oldTime: oldTime = newTime
    ioEnd: int = (ioRead - oldRead)
    timeEnd: float = (time.time() - oldTime)
    if ioEnd > 0:
        if timeEnd < 1: timeEnd = 1;
        diskRead = ioEnd / timeEnd;
    cache.set('io_read', ioRead)
    if diskRead > 0: return int(diskRead)
    return 0


def GetRegValue(key: str, subkey: str, value: str) -> Any:
    '''
    获取系统注册表信息

    Parameters
    ----------
    key : str
        类型.
    subkey : str
        路径.
    value : str
        key.

    Returns
    -------
    value : Any
        DESCRIPTION.

    '''
    import winreg
    key = getattr(winreg, key)
    handle = winreg.OpenKey(key, subkey)
    (value, type) = winreg.QueryValueEx(handle, value)
    return value


def GetSystemVersion() -> str:
    '''
    获取操作系统版本（多平台）

    Returns
    -------
    str
        DESCRIPTIONm.

    '''
    if UNIX: return GetSystemVersionUnix()
    return GetSystemVersionWindows()


def GetSystemVersionWindows() -> str:
    '''
    获取操作系统版本（windows）

    Returns
    -------
    str
        DESCRIPTION.

    '''
    try:
        import platform
        bit: str = 'x86';
        if 'PROGRAMFILES(X86)' in os.environ: bit = 'x64'

        def get(key: str):
            return GetRegValue(
                "HKEY_LOCAL_MACHINE",
                "SOFTWARE\\Microsoft\\Windows NT\\CurrentVersion",
                key
            )

        osName = get('ProductName')
        build = get('CurrentBuildNumber')

        version: str = '{} (build {}) {} (Py{})'.format(
            osName, build, bit, platform.python_version())
        return version
    except Exception as ex:
        print('获取系统版本失败，错误：' + str(ex))
        return '未知系统版本.'


def GetSystemVersionUnix() -> str:
    '''
    获取系统版本（unix）

    Returns
    -------
    str
        系统版本.

    '''
    try:
        version: str = readFile('/etc/redhat-release')
        if not version:
            version = readFile(
                '/etc/issue'
            ).strip().split("\n")[0].replace('\\n','').replace('\l','').strip()
        else:
            version = version.replace(
                'release ',''
            ).replace('Linux','').replace('(Core)','').strip()
        v = sys.version_info
        return version + '(Py {}.{}.{})'.format(v.major, v.minor, v.micro)
    except Exception as err:
        print('获取系统版本失败，错误：', err)
        return '未知系统版本.'


def GetBootTime() -> dict:
    '''
    获取当前系统启动时间

    Returns
    -------
    dict
        DESCRIPTION.

    '''
    bootTime: float = psutil.boot_time()
    return {
        'timestamp': bootTime,
        'runtime': time.time() - bootTime,
        'datetime': time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())
    }


def GetCpuConstants() -> dict:
    '''
    获取CPU常量信息

    Parameters
    ----------
    cpuConstants : CpuConstants
        DESCRIPTION.

    Returns
    -------
    dict
        DESCRIPTION.

    '''
    return cpuConstants.getDict


def GetFullSystemData() -> dict:
    '''
    获取完全的系统信息

    Returns
    -------
    dict
        DESCRIPTION.

    '''
    systemData: dict = {
        **GetSystemInfo(),
        'network': { **GetNetWork() },
        'io': { **GetIoReadWrite() },
        'boot': { **GetBootTime() },
        'time': time.time()
    }
    return systemData

cpuConstants = CpuConstants()

if __name__ == '__main__':
    print(GetFullSystemData())
    print(GetCpuConstants())
    print(GetSystemInfo())
    print(GetNetWork())
    print(GetIoReadWrite())
    


{'cpu': {'used': 10.5, 'used_list': [3.7, 8.9, 14.6, 6.5, 8.5, 1.8, 5.2, 2.2, 2.2, 4.9, 2.7, 1.2, 2.3, 1.3, 5.0, 2.9, 2.4, 6.1, 6.0, 2.4], 'cpu_count': 1, 'cpu_name': 'Intel(R) Core(TM) i9-10900K CPU @ 3.70GHz', 'cpu_core': 10, 'cpu_threads': 20}, 'load': {'one': 0, 'five': 0, 'fifteen': 0, 'max': 40, 'limit': 40, 'safe': 30.0}, 'mem': {'memTotal': 32480, 'memFree': 21799, 'memRealUsed': 10681, 'menUsedPercent': 32.88534911199906}, 'disk': [{'path': 'C:/', 'size': {'total': 508453597184, 'used': 251668832256, 'free': 256784764928, 'percent': 49.5}, 'fstype': 'NTFS', 'inodes': False}], 'network': {'up': 0, 'down': 0, 'upTotal': 83814717, 'downTotal': 390884500, 'downPackets': 466817, 'upPackets': 446254}, 'io': {'write': 0, 'read': 0}, 'boot': {'timestamp': 1687137044.5692391, 'runtime': 90476.80121970177, 'datetime': '2023-06-20 10:18:41'}, 'time': 1687227521.3704588}
{'cpu_count': 1, 'cpu_name': 'Intel(R) Core(TM) i9-10900K CPU @ 3.70GHz', 'cpu_core': 10, 'cpu_threads': 20}
{'cpu': {'

In [ ]:
for mac in c.Win32_NetworkAdapter():
    print("mac地址", mac.MACAddress)

In [17]:
import uuid
import wmi
import socket

class information:
    w = wmi.WMI() # 获取配置信息
    list = []

class INFO(information):
    def __init__(self):
        self.info()

    # 获取配置信息
    def info(self):
    
        for BIOSs in information.w.Win32_ComputerSystem():
            information.list.append("电脑名称: %s" % BIOSs.Caption)
            information.list.append("使 用 者: %s" % BIOSs.UserName)
            
        
        address = hex(uuid.getnode())[2:]
        address =  ":".join([address[e:e+2] for e in range(0,11,2)])
        information.list.append("MAC地址: %s" % address)
        
        for BIOS in information.w.Win32_BIOS():
            # information.list.append("使用日期: %s" % BIOS.Description)
            information.list.append("主板型号: %s" % BIOS.SerialNumber)
            
        for processor in information.w.Win32_Processor():
            information.list.append("CPU型号: %s" % processor.Name.strip())
            
        for memModule in information.w.Win32_PhysicalMemory():
            print('*'*50)
            totalMemSize = int(memModule.Capacity)
            information.list.append("内存厂商: %s" % memModule.Manufacturer)
            information.list.append("内存型号: %s" % memModule.PartNumber.strip())
            information.list.append("内存大小: %.2fGB" % (totalMemSize / 1024**3))
            
        for disk in information.w.Win32_DiskDrive(InterfaceType="IDE"):
            
            diskSize = int(disk.size)
            information.list.append("磁盘名称: %s" % disk.Caption)
            information.list.append("磁盘大小: %.2fGB" % (diskSize / 1024**3))
            
        for xk in information.w.Win32_VideoController():
            information.list.append("显卡名称: %s" % xk.name)


        addr = socket.gethostbyname(socket.gethostname())
        information.list.append("IP地址: %s" % addr)

        return '\n'.join(information.list)

infor = information()
INFOs = INFO()
print(information.list)



**************************************************
**************************************************
['电脑名称: DESKTOP-JHRQTGU', '使 用 者: DESKTOP-JHRQTGU\\Opti7080', 'MAC地址: cc:d9:ac:d8:57:06', '使用日期: 1.19.0', '主板型号: 41JYJ53', 'CPU型号: Intel(R) Core(TM) i9-10900K CPU @ 3.70GHz', '内存厂商: 80AD000080AD', '内存型号: HMA82GU6DJR8N-XN', '内存大小: 16.00GB', '内存厂商: 80AD000080AD', '内存型号: HMA82GU6DJR8N-XN', '内存大小: 16.00GB', '显卡名称: NVIDIA GeForce RTX 2070 SUPER', '显卡名称: Intel(R) UHD Graphics 630', 'IP地址: 10.176.130.235']


In [15]:
print()

电脑名称: DESKTOP-JHRQTGU
使 用 者: DESKTOP-JHRQTGU\Opti7080
MAC地址: cc:d9:ac:d8:57:06
使用日期: 1.19.0
主板型号: 41JYJ53
CPU型号: Intel(R) Core(TM) i9-10900K CPU @ 3.70GHz
内存厂商: 80AD000080AD
内存型号: HMA82GU6DJR8N-XN
内存大小: 16.00GB
内存厂商: 80AD000080AD
内存型号: HMA82GU6DJR8N-XN
内存大小: 16.00GB
显卡名称: NVIDIA GeForce RTX 2070 SUPER
显卡名称: Intel(R) UHD Graphics 630
IP地址: 10.176.130.235


In [2]:
from datasets import load_dataset

dataset = load_dataset("BAAI/COIG")
#BAAI/COIG  FreedomIntelligence/phoenix-sft-data-v1 silk-road/Wizard-LM-Chinese-instruct-evol

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 282.25it/s]
Generating train split: 0 examples [00:00, ? examples/s]Failed to read file 'C:\Users\Opti7080\.cache\huggingface\datasets\downloads\f333eaf7bddee100eed8b350de14d70471681ce0d62c310589a8464d46c13a29' with error <class 'pyarrow.lib.ArrowInvalid'>: JSON parse error: Column(/textbox_q_context) changed from string to array in row 3


DatasetGenerationError: An error occurred while generating the dataset

In [59]:
dataset['train']

Dataset({
    features: ['conversations', 'id'],
    num_rows: 464510
})

In [50]:
dataset['train'][190000]['id']

'[190284] [Type: Instruction] [Lang: multi-lingual] [Dataset: Alpaca-gpt4-post-translation]'

In [24]:
if dataset['train'][1]['output_zh']:
    print(1)

In [15]:
import pandas as pd
import numpy as np

output_list = pd.DataFrame()
output_list['instruction'] = np.nan

In [28]:
dataset['train'][0]

{'id': 1635,
 'category': 'information_extraction',
 'instruction_zh': '根据以下关于瑞典经济的文章，哪个经济部门占据了最大的产出？',
 'context_zh': '瑞典是一个出口导向的混合经济体，拥有现代化的分销系统、优秀的内外部通讯和熟练的劳动力。木材、水力和铁矿石构成了一个经济体的资源基础，这个经济体严重依赖对外贸易。瑞典的工程部门占产出和出口的50%。电信、汽车工业和制药工业也非常重要。农业占GDP和就业的2%。军火工业具有高度先进的技术声誉。',
 'response': 'According to this passage, the engineering sector accounts for the largest output, generating 50% of output and exports.',
 'instruction': 'Based on the following passage regarding the economy of Sweden, what is the economic sector that accounts for the largest output?',
 'context': "Sweden is an export-oriented mixed economy featuring a modern distribution system, excellent internal and external communications, and a skilled labor force. Timber, hydropower and iron ore constitute the resource base of an economy heavily oriented toward foreign trade. Sweden's engineering sector accounts for 50% of output and exports. Telecommunications, the automotive industry and the pharmaceutical industries are also of gre

In [13]:
import re

def has_chinese(text):
    pattern = re.compile(r'[\u4e00-\u9fa5]')  # 匹配中文字符的正则表达式
    match = re.search(pattern, text)
    if match:
        return True
    else:
        return False

# 示例用法
sentence1 = "shij!"  # 包含中文字符
print(has_chinese(sentence1))  # 输出: True

sentence2 = "Hello, World!"  # 不包含中文字符
print(has_chinese(sentence2))  # 输出: False


False
False


In [3]:
from datasets import load_dataset

dataset = load_dataset("silk-road/Wizard-LM-Chinese-instruct-evol")

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 83.97it/s]


Dataset json downloaded and prepared to C:/Users/Opti7080/.cache/huggingface/datasets/silk-road___json/silk-road--Wizard-LM-Chinese-instruct-evol-e6be7ef9d3a16fba/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 44.16it/s]


In [ ]:
messageshistory = [
                    {"role":"system","content":"You are an AI assistant that helps people find information."},
                    {"role":"user","content":"请给我介绍一下联想企业"},
                    ]
response = openai.ChatCompletion.create(
      engine="Kepler1",
      messages = messageshistory,
      temperature=0.5,
      max_tokens=300,
      top_p=0.95,
      frequency_penalty=0,
      presence_penalty=0,
      stop=None)
print('AI bot:',response['choices'][0]['message']['content'])
bot = response['choices'][0]['message']['content']

In [ ]:



# print(response)
# print(response['choices'][0]['message']['content'])

while True :
    response = openai.ChatCompletion.create(
      engine="Kepler1",
      messages = messageshistory,
      temperature=0.5,
      max_tokens=800,
      top_p=0.95,
      frequency_penalty=0,
      presence_penalty=0,
      stop=None)
    print('AI bot:',response['choices'][0]['message']['content'])  # print the reply of the bot.
    bot = response['choices'][0]['message']['content']
    a = 1

    messageshistory.append({"role":"assistant","content":bot}) # add bot output to history for chatbot to understand and respond
    i = input('User: ')
    messageshistory.append({"role":"user","content":i}) # add user input to history

In [62]:
opt5 = Operation5('请帮我降低音量，',model_sim,tokenizer_sim)
opt5.judge()

音量功能匹配为调节音量


In [46]:
opt5.corpus_embeddings,opt5.input_embeddings

(array([[-0.03121485,  0.00247283, -0.03576345, ..., -0.00866181,
          0.02807012,  0.04373301],
        [-0.02591462, -0.01915512, -0.04096009, ..., -0.02056012,
          0.00361972,  0.01807105]], dtype=float32),
 array([372], dtype=int64))

In [52]:
(opt5.corpus_embeddings@opt5.input_embeddings.T).argmax(axis=0)[0]

1

In [17]:
volume.SetMute(0, None)

0

In [16]:
volume.SetMute(1, None)

1

In [1]:
volume.SetMasterVolumeLevel(-65.25 + 8.26, None)

NameError: name 'volume' is not defined

In [2]:
import ctypes
import tkinter as tk
from tkinter import ttk
from pycaw.pycaw import AudioUtilities, ISimpleAudioVolume

# 创建一个Tkinter窗口
window = tk.Tk()

window.title("音量控制")
window.geometry("300x100")

# 获取默认的音频会话管理器
sessions = AudioUtilities.GetAllSessions()
for session in sessions:
    volume = session._ctl.QueryInterface(ISimpleAudioVolume)
    if session.Process and session.Process.name() == "python.exe":
        volume.SetMasterVolume(0.5, None)  # 初始化音量为50%

# 函数：调节音量
def set_volume(vol):
    for session in sessions:
        volume = session._ctl.QueryInterface(ISimpleAudioVolume)
        if session.Process and session.Process.name() == "python.exe":
            volume.SetMasterVolume(vol / 100, None)

In [3]:
import os
os.system('ffmpeg -y -i data/voice1.wav -ac 1 -ar 16000 data/voice.wav')

1

In [1]:
import socket 


In [2]:
import uuid

def get_mac_address():
    mac=uuid.UUID(int = uuid.getnode()).hex[-12:]
    return ":".join([mac[e:e+2] for e in range(0,11,2)])

print(get_mac_address())

cc:d9:ac:d8:57:06


In [8]:
import socket
def get_computer_name_ip():
    #获取本机电脑名
    name = socket.getfqdn(socket.gethostname())
    #获取本机ip
    addr = socket.gethostbyname(socket.gethostname())
    return name,addr

myname,myaddr = get_computer_name_ip()
print('电脑名:',myname)
print('ip地址:',myaddr)


电脑名: DESKTOP-JHRQTGU.lenovo.com
ip地址: 10.176.130.235


In [9]:
import wmi

c = wmi.WMI()

# # 硬盘序列号
for physical_disk in c.Win32_DiskDrive():
    print("硬盘序列号", physical_disk.SerialNumber)

# CPU序列号
for cpu in c.Win32_Processor():
    print("CPU序列号", cpu.ProcessorId.strip())

# 主板序列号
for board_id in c.Win32_BaseBoard():
    print("主板序列号", board_id.SerialNumber)

# mac地址
for mac in c.Win32_NetworkAdapter():
    print("mac地址", mac.MACAddress)

# bios序列号
for bios_id in c.Win32_BIOS():
    print("bios序列号", bios_id.SerialNumber.strip())



硬盘序列号 E823_8FA6_BF53_0001_001B_444A_4423_4BF2.
CPU序列号 BFEBFBFF000A0655
主板序列号 /41JYJ53/CNFCW0007K00LH/
mac地址 None
mac地址 CC:D9:AC:D8:57:02
mac地址 A4:BB:6D:CE:5A:8C
mac地址 CC:D9:AC:D8:57:06
mac地址 CC:D9:AC:D8:57:03
mac地址 None
mac地址 None
mac地址 None
mac地址 None
mac地址 None
mac地址 BC:9B:20:52:41:53
mac地址 BA:7F:20:52:41:53
mac地址 BA:8A:20:52:41:53
mac地址 CE:D9:AC:D8:57:02
bios序列号 41JYJ53


In [ ]:
import os
